In [229]:
import pyodbc
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.metrics import precision_score
import pickle
from datetime import datetime
import warnings

def getData(symbol):
    conn = pyodbc.connect(
        "DRIVER={SQL Server};SERVER=lenovo-desktop;DATABASE=Qihuo;UID=samtsql;PWD=F(W}q:TsyK,7^+>`P28e79s#Uc5n")
    query = f"""select m1.date as 'Date',DATEPART(hour, m1.Date) AS 'Hour', m1.[Close] as 'Market', m2.[close] as 'Stock'  from MinuteQuote m1
    inner join MinuteQuote m2 on m1.Date = m2.date
    where m1.Date > '2002-04-08 01:20:00' and m1.Contract = 'ym' and m2.Contract = '{symbol}'

    order by m1.date"""

    with warnings.catch_warnings():
        warnings.simplefilter('ignore', UserWarning)
        df = pd.read_sql(query, conn)
        df.set_index('Date', inplace=True)
        return df

 


In [230]:
def transformData(df):
    df["StockReturn"] = df.rolling(2).apply(lambda x: x.iloc[1] / x.iloc[0] - 1)["Stock"]
    df["MarketReturn"] = df.rolling(2).apply(lambda x: x.iloc[1] / x.iloc[0] - 1)["Market"]
    df["OutPerform"] = df["StockReturn"] - df["MarketReturn"]
    df["Target"] = (df.apply(lambda x: x > 0)["OutPerform"]).astype(int)


In [231]:
def getPredictors(df):
    predictors = ['Hour']
    for i in range(12):
        df[f'OutPerform{pow(2,i)*5}'] = df['OutPerform'].rolling(pow(2,i)).sum()
        predictors.append(f'OutPerform{pow(2,i)*5}')
        df[f'StockReturn{pow(2,i)*5}'] = df['StockReturn'].rolling(pow(2,i)).sum()
        predictors.append(f'StockReturn{pow(2,i)*5}')
        # df[f'MarketReturn{i}'] = df["Market"] / df["Market"].shift(pow(2,i)) - 1
        # predictors.append(f'OutPerform{i}')
        # df[f'StockReturn{i}'] = df["Stock"] / df["Stock"].shift(pow(2,i)) - 1
        # predictors.append(f'StockReturn{i}')
        # df[f"OutPerform{i}"] = df[f'StockReturn{i}'] - df[f'MarketReturn{i}']
        # df.drop([f'MarketReturn{i}'], axis = 1)
    return predictors


In [232]:
def getMLdata(df, predictors):
    df = df[df.index.minute % 30 == 0]
    prev = df.copy()
    prev = prev.shift(1)
    data = df[["OutPerform","Target","Stock","Market"]]
    data = data.join(prev[predictors])
    transformData(data)
    data = data.copy().dropna()
    # print(data)
    return data


In [233]:
def predict(train, test, predictors, model):
    model.fit(train[predictors], train["Target"])
    preds = model.predict_proba(test[predictors])[:,1]
    preds = pd.Series(preds, index=test.index, name="Predictions")
    combined = pd.concat([test["Target"], preds], axis=1)
    return combined

In [234]:
def backtest(data, model, predictors, start=20000, step=5000):
    all_predictions = []
    loop = 0
    for i in range(start, data.shape[0], step):
        train = data.iloc[loop * step:i].copy()
        test = data.iloc[i:(i+step)].copy()
        predictions = predict(train, test, predictors, model)
        all_predictions.append(predictions)
        loop = loop + 1

    # print(all_predictions)    
    return pd.concat(all_predictions)


In [235]:
model = RandomForestClassifier(n_estimators=100, min_samples_split=1200, random_state=1)
symbols = [
'CVX'
,'HON',
'CRM',
'UNH',
'CSCO',
'WMT',
'AXP',
'JPM',
'MCD',
'HD',
'AMGN',
'V',
'INTC',
'WBA',
'GS',
'JNJ',
'PG',
'AAPL',
'DIS',
'MMM',
'MRK',
'MSFT',
'TRV',
'VZ',
'IBM',
'CAT',
'NKE'
]
for s in symbols:
    df = getData(s)
    transformData(df)
    predictors = getPredictors(df)
    data = getMLdata(df, predictors)
    predictions = backtest(data, model, predictors)
    pickle.dump(model, open(f"Model/{s}", 'wb'))
    tempData = data[['OutPerform']].copy()
    result = pd.merge(tempData, predictions, left_index=True, right_index=True)
    result.to_csv(f'./Result/{s}.csv')

    # precision_score(predictions["Target"], predictions["Predictions"])
    

In [284]:
import glob
files = [f for f in glob.glob("Result/*.csv")]
result = pd.DataFrame(columns = ['symbol', 'year', 'gain','count'])
for file in files:

    s = file.split('\\')[1].replace('.csv','')
    df = pd.read_csv(f'Result/{s}.csv')
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    df['year'] = df.index.strftime('%Y')
    groupbyyear = df.groupby('year')

    for group_name, df_group in groupbyyear:
        count = 0
        newtrade = True
        gsum = 0
        for ind in df_group.index:
            if df_group['Predictions'][ind] > .57:
                gsum = gsum + df_group['OutPerform'][ind]
                if newtrade == True:
                    count = count + 1
                    newtrade = False
            elif df_group['Predictions'][ind] >= .50 and newtrade == False:
                gsum = gsum + df_group['OutPerform'][ind]
            else:
                newtrade = True 

        if count > 0:       
            result.loc[len(result.index)] = [s, group_name, gsum, count]

dfgain = result.groupby('year')['gain'].sum()
dfcount = result.groupby('year')['count'].sum()
dfresult = dfgain.to_frame().merge(dfcount.to_frame(), left_index=True, right_index=True)
dfresult['avg'] = dfresult['gain']/dfresult['count']
print(dfresult)
print(dfresult['count'].sum())
print(dfresult['gain'].sum()/dfresult['count'].sum())            

          gain  count       avg
year                           
2008 -6.166510   3119 -0.001977
2009 -2.457576   3134 -0.000784
2010 -1.777685   1928 -0.000922
2011 -1.958083   1887 -0.001038
2012 -1.152642   1489 -0.000774
2013 -0.788516   1189 -0.000663
2014 -0.174894    865 -0.000202
2015 -0.777353   1063 -0.000731
2016 -0.439312    937 -0.000469
2017 -0.388631    645 -0.000603
2018 -0.962799    930 -0.001035
2019 -0.215986    743 -0.000291
2020 -1.557543    986 -0.001580
2021 -0.182617    413 -0.000442
2022 -0.233668    258 -0.000906
19586
-0.000982018633673552


In [237]:
dfgain = result.groupby('symbol')['gain'].sum()
dfcount = result.groupby('symbol')['count'].sum()
dfresult = dfgain.to_frame().merge(dfcount.to_frame(), left_index=True, right_index=True)
dfresult['avg'] = dfresult['gain']/dfresult['count']
print(dfresult)
print(dfresult['count'].sum())
print(dfresult['gain'].sum()/dfresult['count'].sum())  

            gain  count       avg
symbol                           
AAPL   -1.449753   2464 -0.000588
AMGN   -0.237136    430 -0.000551
AXP    -0.935728    725 -0.001291
CAT    -0.519676    546 -0.000952
CRM    -0.425134    103 -0.004128
CSCO   -1.685275   1654 -0.001019
CVX    -0.571887    252 -0.002269
DIS    -1.066505    824 -0.001294
GS      0.018812    536  0.000035
HD     -0.296671    235 -0.001262
HON    -1.287991   1270 -0.001014
IBM    -1.201634   1047 -0.001148
INTC   -1.384502   2030 -0.000682
JNJ    -0.339659    423 -0.000803
JPM    -0.641054    728 -0.000881
MCD    -0.112153    164 -0.000684
MMM    -1.544818   1050 -0.001471
MRK    -0.091216    313 -0.000291
MSFT   -0.595839   1180 -0.000505
NKE    -0.473535    650 -0.000729
PG     -0.155105    552 -0.000281
TRV    -1.202037    806 -0.001491
UNH    -0.842941    297 -0.002838
V      -0.202276    332 -0.000609
VZ     -0.329904    553 -0.000597
WBA    -0.585490    501 -0.001169
WMT    -0.255519    619 -0.000413
20284
-0.00090

In [276]:

result

,symbol,year,gain,count
0,AAPL,2008,0.435035,246
1,AAPL,2009,0.619470,236
2,AAPL,2010,0.254073,137
3,AAPL,2011,0.132342,91
4,AAPL,2012,-0.007467,89
...,...,...,...,...
357,WMT,2017,0.044086,3
358,WMT,2018,0.004285,2
359,WMT,2019,-0.000829,1
360,WMT,2020,0.018254,9
